<a href="https://colab.research.google.com/github/AndresMontesDeOca/RegresionAvanzada/blob/main/Machete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MACHETE

#### Logbook

In [351]:
# Autor: Andres Montes de Oca

# 31/05/23 -> Creacion de la Notebook
# 31/05/23 -> Tests Normalidad, Homocedasticidad Residuos
# 06/06/23 -> Test No-autoorrelacion de Residuos
# 17/06/23 -> Transfomraciones Box Clox
# 19/06/23 -> Migrated to Google Colab
# 20/06/23 -> R Magic
# 22/06/23 -> Deteccion Outliers e Influyentes

In [ ]:
# Instalacion de Paquetes
!pip install pingouin # No incluido en Google Colab
# !pip install scipy
# !pip install statsmodels

# Cargamos Librerias y Datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
from scipy import stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tools.tools as smt

# Ignorar Warnings
import warnings
warnings.filterwarnings('ignore')

# Asthetics
sns.set(style='ticks', context='notebook', palette='colorblind', font_scale=1, color_codes=True)

# Recursion limit errors with R Magic
import sys
# sys.setrecursionlimit(50000)

# Activamos R magic
%load_ext rpy2.ipython

### Libreria rpy2 (Python <==> R)
- No la usamos ya que en Colab disponemos de R Magic

In [353]:
# Version rpy2 que no tiene problemas de compatibilidad
# !pip install rpy2==3.5.1

# Import rpy2 for dataframe conversion
# import rpy2.robjects as ro
# from rpy2.robjects.packages import importr
# from rpy2.robjects import pandas2ri
# from rpy2.robjects.conversion import localconverter
# from rpy2.robjects import globalenv


# ###### Pandas DataFrames and Series conversion ########
# # Cargamos un DataSet cualquira en Python, para que no de error
# data_P = sns.load_dataset('iris')
# Serie = data_P['petal_length']

# # Convert the Python DataFrame to the R dataframe
# %R -i data_P

# with localconverter(ro.default_converter + pandas2ri.converter):
#   data_R = ro.conversion.py2rpy(data_P)
# # Create a variable name in R's Global Environment
# globalenv['data_R'] = data_R

# # Convert Python Series to R vectors
# vec_float_R = ro.vectors.FloatVector(Serie)
# # vec_int_R = ro.vectors.IntVector(Serie)
# # vec_str_R = ro.vectors.StrVector(Serie)
# globalenv['vec_float_R'] = vec_float_R

# # Convert R datadrame/vector to Python DataFrame/Vector
# %R -o data_R

# with localconverter(ro.default_converter + pandas2ri.converter):
#   data_P = ro.conversion.rpy2py(data_R)

# # Importar R-Functions a Python como Objetos(sin uso)
# shapiro_test = ro.r('shapiro.test')
# result = shapiro_test(vec_float_R)

#### Instalaciones Individuales en Bash

In [354]:
# system(sudo apt install libgsl-dev) # -> Rapido

#### Instalacion de Librerias para R en Google Colab

In [355]:
%%R

### MVN Henze-Zirkler Test ### -> Lento
# # system(sudo apt install libgsl-dev)
# install.packages('MVN')

# ## Wald Test ### -> Rapido
# install.packages('aod')

# ## QQ Plot, Bonferroni ### -> Lento
# install.packages('car')

# # LRM Tests ### -> Rapido
# install.packages('lmtest')

# ## Box-Cox Transformation ### -> Rapido
# install.packages('MASS')


####################################
### Installation Status ###
# print(system.file(package='MVN'))

NULL


## Cargas de Datasets from Google Drive

In [356]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

################################# Datasets ###################################
# Grasa de Cerdos
id = '153lGVzdixcHT-keKg8qmvaoWdPHg6_tB'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('grasacerdos.xlsx')
dataG = pd.read_excel('grasacerdos.xlsx', index_col='Obs') # Cargamos el Dataset
dataG = dataG.replace(to_replace=',', value='.', regex=True) # Reemplazo , por .
dataG = dataG.astype('float') # Transformo en float
%R -i dataG

######################################################
# Peso, Edad, Colesterol
id = '17Dv1WcWlc9ojWa6bnSfD7TCRaVKCKqj0'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('peso_edad_colest.xlsx')
dataP = pd.read_excel('peso_edad_colest.xlsx')
%R -i dataP

# # Convert the Python DataFrame to the R dataframe
# with localconverter(ro.default_converter + pandas2ri.converter):
#   dataP_R = ro.conversion.py2rpy(dataP)
# # Create a variable name in R's Global Environment
# globalenv['dataP_R'] = dataP_R

######################################################
# Cars con Modelo Lineal
id = '15pf-6P4Ek2rp6mYmfOybHRqAxqOoTp6i'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('cars.csv')
dataC = pd.read_csv('cars.csv')
model_cars = smf.ols('dist ~ speed', data=dataC).fit()
%R -i dataC

######################################################
# University
id = '16kLQNXhyweAi38xR2IeQCCagVcU725UZ'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('University.csv')
dataU = pd.read_csv('University.csv')
%R -i dataU

######################################################
# Iris(Python)
dataI = sns.load_dataset('iris')

# # Iris(R) -> DataFrame to dataframe
# with localconverter(ro.default_converter + pandas2ri.converter):
#   dataI_R = ro.conversion.py2rpy(dataI.drop(columns='species'))
# # Create a variable name in R's global environment
# globalenv['dataI_R'] = dataI_R

# # Serie to Vector
# x_R = ro.FloatVector(dataI['petal_length'])
# w_R = ro.FloatVector(dataI['petal_width'])
# globalenv['x_R'] = x_R
# globalenv['w_R'] = w_R
%R -i dataI

######################################################
# Gorriones
id = '15G6jIOMiuWaTs-qy7eUXwhED4delBMl6'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('gorriones.xlsx')
dataO = pd.read_excel('gorriones.xlsx')
dataO.rename(columns=
            {'largototal':'Largo', 'extension ':'Alas', 'cabeza':'Cabeza',
             'humero':'Pata', 'esternon':'Cuerpo', 'sobrevida ':'Target'},
            inplace=True)
%R -i dataO


# Analisis de Variables

In [377]:
# data=dataI

# # Relacion Lineal Grafica
# sns.scatterplot(data=data, x='petal_length', y='petal_width')
# plt.show()

# # Multicollonearity - Eeach independent variable should be independent from other independent variables
# print(pg.corr(data['petal_length'], data['petal_width'], method='pearson')) # Entre dos varibales, H0 -> Variables Independientes
# print(pg.corr(data['petal_length'], data['petal_width'], method='spearman')) # Cuando no se cumplen los supuestos de Pearson
# display(sns.heatmap(data.corr(), vmin=-1, vmax=1, cmap='RdYlGn', annot=True)) # Todos contra todos

# # Analsis de Noramlidad Multi-Variariable (Henze-Zirkler) -> Python
# print(pg.multivariate_normality(data.drop(columns='species'))) # Henze-Zirkler Test, H0 -> Multiv Normal Dist

# # Analsis de Noramlidad Multi-Variariable (Henze-Zirkler) -> R
# # library(MVN)
# # mvn_result <- mvn(data, mvnTest = 'hz')
# # print(mvn_result)

# Linear Regression Models

In [386]:
# # En R

# %%R
# data <- dataP

# model <- lm('colest ~ edad', data=data)
# resid = model$resid
# predicted = predict(model)
# fitted = model$fitted.values
# # print(summary(model_colest_R))
# # print(anova(model_colest_R))

# # Bandas de Prediccion
# predichos <- predict(object=model, interval='prediction', level=0.95)
# nuevos_datos <- data.frame(predichos, data)

# head(nuevos_datos)

In [387]:
# # Graficamos en Python
# %R -o nuevos_datos

# # Grafico
# sns.regplot(data=nuevos_datos, x='edad', y='colest')
# sns.lineplot(data=nuevos_datos, x='edad', y='lwr', linestyle='--', color='r')
# sns.lineplot(data=nuevos_datos, x='edad', y='upr', linestyle='--', color='r')
# plt.show()

In [388]:
# # LRM en Python
# data = dataP

# # Generamos el Modelo
# data = sm.add_constant(data)
# model = smf.ols('colest ~ edad', data=data).fit()

# # Ploteamos
# sns.regplot(data=data, x='edad', y='colest')
# plt.show()

### Wald Test (revisar)
Mide si la Variable X es lo suficientemente significativa como para explicar la

In [399]:
# #####   PENDIENTE    ######
# %%R
# data <- dataO

# library(aod)

# # Generamos el Modelo
# model_gorr = lm('Largo ~ Alas + Cabeza + Pata + Cuerpo', data=data)
# print(summary(model_gorr))

# # Wald Test para indentificar variables significativas
# wald.test(Sigma = vcov(model_gorr), b = coef(model_gorr), Terms = 4) # Cuerpo| H0: Coef = 0 (Var no significativa)


# Analisis de Diagnostico

### Residuals Nomrality

In [401]:
# %%R

# data <- dataI

# # Analisis de Normalidad en los Residuos
# library(car)
# qqPlot(resid) # Observaciones que romperian con el supuesto de Normalidad
# print(shapiro.test(resid))

In [403]:
# # Python
# data = dataI

# # Tests Analiticos
# display(pg.normality(data['sepal_width'])) # Shapiro Test(single Var), H0 -> Normal Dist
# print(st.anderson(dataI['sepal_width'], dist='norm')) # Anderson-Darling SciPy
# print(st.kstest(dataI['sepal_width'], 'norm'))# Kolmogorov-Smirnov SciPy

# # Tests Graficos
# pg.qqplot(data['sepal_width'])
# plt.show()

### Residuals Variance (Homosedasticity)

In [405]:
# # Analisis Analitico (Breusch-Pagan Test) -> R | H0: Residuos Homocedasticos
# %%R

# library(lmtest)
# bptest(model)

In [407]:
# # # Analsis Grafico -> Python

# # Importo Datos desde R
# %R -o resid
# %R -o predicted
# %R -o fitted

# plt.scatter(x=predicted, y=resid)
# plt.xlabel( 'Prediccion')
# plt.ylabel('Residuo')
# plt.title('Distribucion de Residuos')
# plt.axhline(color='grey', linestyle='dashed', alpha=0.5)
# plt.show()
# # No se observa estructura de embudo

# # # Analisis Analitico Python
# BP, p_value, _, _ =sm.stats.het_breuschpagan(model.resid, model.model.exog)
# print('Estadistico BP y p-value:', BP, p_value)

### Residuals Auto-Correlation

In [412]:
# # Analisis Analitico (Durbin-Watson Test) -> R | H0 No Auto-Correlacion
# # 2=No Correlacion (Independecia)| 0=Correlacion Pos | 4=Correlacion Neg
# %%R

# library(lmtest)
# dwtest(model, alternative='two.sided', iterations=1000)

In [413]:
# # Analsis Grafico -> Python
# data=dataP
# plt.scatter(x=data.index, y=resid)
# plt.xlabel( 'Index')
# plt.ylabel('Residuo')
# plt.title('Correlacion de Residuos')
# plt.axhline(color='grey', linestyle='dashed', alpha=0.5)
# plt.show()
# # No se observa estructura

# # Analisis Analitico -> Python
# print('Durbin-Watson:', sm.stats.durbin_watson(resid)) # Sin validacion Estadistica

# Transformaciones
- Cuando los residuos no siguen una distribucion Normal -> Transformamos la y (BoxCox)
- Cuando los residuos tienen estructura -> Transformamos/Agregamos X

In [427]:
# # Box Cox Transformations -> R
# %%R
# data = dataC
# library(MASS)

# # Generamos el modelo y verificamos la Normalidad de los residuos
# model <- lm('dist ~ speed', data=data)
# print(shapiro.test(model$resid))

# # Buscamos el Lambda  optimo -> Si Lambda  = 0 -> log10(y) | y**Lambda
# boxcox(object = model, plotit=TRUE)

In [437]:
# # # Transformamos/Agregamos Variables en Dataset original -> Python
# data = dataC

# dist_p2 = pd.Series(data['dist']**2, name='dist_p2')
# dist_log10 = pd.Series(np.log10(data['dist']), name='dist_log10')

# dataC2 = pd.concat([data, dist_p2, dist_log10], axis=1)
# display(dataC_2.head())

# %R -i dataC2 # Importamos a R

In [436]:
# # # Generamos el nuevo modelo con las nuevas variables transformadas, y veridicamos su Normalidad
# %%R

# model_2 <- lm('dist_p2 ~ speed', data=dataC2)
# print(shapiro.test(model_2$resid))

# # Bandas de Prediccion
# predichos <- predict(object=model_2, interval='prediction', level=0.95)
# nuevos_datos <- data.frame(predichos, dataC2)

# head(nuevos_datos)

In [435]:
# # Graficamos con las Bandas de Confianza y Prediccion
# %R -o nuevos_datos # Exportamos desde R

# sns.regplot(data=nuevos_datos, x='speed', y='dist_p2')
# sns.lineplot(data=nuevos_datos, x='speed', y='lwr', linestyle='--', color='r')
# sns.lineplot(data=nuevos_datos, x='speed', y='upr', linestyle='--', color='r')
# plt.show()

# 7) Deteccion de Valores Outliers e Influyentes

In [371]:
# # No Outliers tests found in Python
%%R

# library(car)

# # Outliers - Bonferroni
# qqPlot(residuos)
# outlierTest(model_colest_R)
# influenceIndexPlot(model_colest_R, vars='Bonf')

# # Influyentes - Leverage
# crit_leverage <- 3*mean(hatvalues(model_colest_R))
# leverage <- hatvalues(model_colest_R) > crit_leverage
# print(sum(leverage))
# print(cbind(hatvalues(model_colest_R), crit_leverage, leverage))
# hist(hatvalues(model_colest_R))

# # Influyentes - Cook
# print(cooks.distance(model_colest_R))
# influenceIndexPlot(model_colest_R, vars='Cook')

# # Influyentes - DFFITS (revisar indices en plot)
# p <- length(model_colest_R$coefficients)
# n <- NROW(model_colest_R$residuals)
# dffits_crit <- 2*sqrt(p/n)
# dffits <- dffits(model_colest_R)
# data_dffits <- data.frame(dffits=dffits)


# # Influyentes - DFBetas
# dfbetas_crit <- 1 # Umbral estandard
# dfbetas(model_colest_R)[,2]>dfbetas_crit

# # Resumen
# plot(model_colest_R)
# summary(influence.measures(model=model_colest_R))
# influencePlot(model = model_colest_R)


NULL


In [372]:
# # Exporto a Python y grafico
# %R -o data_dffits
# %R -o dffits_crit

# data_dffits.head()
# sns.scatterplot(data=data_dffits, x=data_dffits.index, y='dffits')
# plt.axhline(y=dffits_crit, linestyle='--', color='red')
# plt.axhline(y=-dffits_crit, linestyle='--', color='red')

# plt.show()

# 9) Transformaciones
- Cuando los residuos no siguen una distribucion Normal -> Transformamos la y (BoxCox)
- Cuando los residuos tienen estructura -> Transformamos/Agregamos en X

In [373]:
# Las transfomraciones de variable las hago en Python

nassets_log10 = pd.Series(np.log10(data['nassets']), name='nassets_log10')
stfees_log10 = pd.Series(np.log10(data['stfees']), name='stfees_log10')
nassets_logE = pd.Series(np.log(data['nassets']), name='nassets_logE')
stfees_logE = pd.Series(np.log(data['stfees']), name='stfees_logE')

data = pd.concat([data, nassets_log10, stfees_log10, nassets_logE, stfees_logE], axis=1)
%R -i data # Importamos a R

KeyError: ignored

In [ ]:
# # Boc Cox Transformation Python
# # Verificamos la Normalidad de los residuos
# print(st.shapiro(model_cars.resid))
# data = dataC
# y = data['dist']

# # Calculo Lambda
# maxlog = st.boxcox(y)[1]
# st.boxcox_normplot(y, -2, 2, plt)
# plt.axvline(maxlog, color='red')
# plt.show()

# # Transfomracion: Si Lambda  = 0 -> np.log10(y) | y**Lambda
# y_trans = pd.Series(y**.5, name='y_trans')
# # No olvidarse de anti-transformar para los IC(10**predicted)

# # Generamos el nuevo Dataset con la y transformada
# data = pd.concat([data, y_trans], axis=1)

# # Generamos el nuevo modelo con la y_transformada
# model_cars_trans = smf.ols('y_trans ~ speed', data=data).fit()

# # Verificamos la nueva normalidad de los residuos
# print(pg.normality(model_cars_trans.resid))

# 10) Cuadrados Minimos Ponderados (WLS)
- Cuando los residuos son Heterocedasticos
- Se compensa la diferencia en los residuos, asignandole diferentes pesos a las observaciones

In [ ]:
# # Cuadrados Minimos Ponderados (WLS) en R
# %%R

# # Generamos el modelo y verificamos la Heterocedasticidad de los residuos analiticamente
# ols_inspec <- lm('inspectores ~ choferes', data=dataCh)
# print(bptest(ols_inspec))

# # Confirmamos con el analisys grafico en Python

# # Generamos la ponderacion de pesos y modelamos de nuevo
# pesos = I(1/ols_onspec$fitted.values**2) # A los valores mas alejados se le reduce el peso
# pesos2 = 1 / lm(abs(ols_inspec$resid) ~ ols_inspec$fitted.values)$fitted.values**2 # Funciona mejor
# pesos3 = 1 / (abs(ols_inspec$resid))#**2

# # Modelamos con lo weigths
# wls_inspec <- lm('inspectores ~ choferes', data=dataCh, weights=pesos)
# print(summary(wls_inspec))

# # Verificar como graficar los modelos W en Python